In [3]:
pip install spacy

Defaulting to user installation because normal site-packages is not writeable
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached wasabi-1.1.3-py3-none-any.whl.metadata (28 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached weasel-0.4.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached typer-0.12.3-py3-none-any.whl.metadata (15 kB)
  Using cached langcodes-3.4.0-py3-none-any.whl.metadata (29 kB)
  Using cached language_data-1.2.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached confection-0.1.5-py3-none-any.whl.metadata (19 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached cloudpathlib-0.18.1-py3-none-any.whl.metadata (14 kB)
   ---------------------------------------- 0.0/12.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/12.1 MB 4.3 MB/s eta 0:00:03
   -- -------------------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [15]:
import pandas as pd
import numpy as np
import spacy
from scipy.sparse import hstack
import string
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier

In [17]:
data = pd.read_csv("spam.csv")
display(data[0:2])

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...


In [18]:
mapping={'ham':0,'spam':1}
data['Category']=data['Category'].map(mapping)

In [13]:
!python -m spacy download en
!pip3 install en_core_web_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)
[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use
the full pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement en_core_web_sm (from versions: none)
ERROR: No matching distribution found for en_core_web_sm


In [15]:
import spacy
spacy.cli.download('en_core_web_sm')

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [19]:
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
punctuations = string.punctuation

In [20]:
def spacy_tokenizer(sentence):
    if type(sentence) == float:
        sentence=sentence
    else:
        doc = nlp(sentence)
        mytokens = [ word.lemma_.lower().strip() for word in doc ]
        mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
        sentence = " ".join(mytokens)
    return sentence

In [21]:
data['tokenized_Message'] = data['Message'].apply(spacy_tokenizer)
display(data[0:2])

,Category,Message,tokenized_Message
0,0,"Go until jurong point, crazy.. Available only ...",jurong point crazy .. available bugis n great ...
1,0,Ok lar... Joking wif u oni...,ok lar ... joke wif u oni ...


In [28]:
x = data.iloc[:,2]
y = data.iloc[:,0]

In [26]:
vec = CountVectorizer()
X_vec = vec.fit_transform(x)
print(X_vec[0:2])

  (0, 3898)	1
  (0, 5276)	1
  (0, 2118)	1
  (0, 1234)	1
  (0, 1628)	1
  (0, 3258)	1
  (0, 7459)	1
  (0, 4007)	1
  (0, 1626)	1
  (0, 1885)	1
  (0, 1036)	1
  (0, 7279)	1
  (1, 4915)	1
  (1, 4040)	1
  (1, 3863)	1
  (1, 7377)	1
  (1, 4940)	1


In [29]:
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [30]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [31]:
clf = Pipeline([('vectorizer', CountVectorizer()),('nb', MultinomialNB())])
clf.fit(x_train,y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [32]:
y_pred=clf.predict(x_test)

In [36]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       966
           1       0.92      0.90      0.91       149

    accuracy                           0.98      1115
   macro avg       0.95      0.94      0.95      1115
weighted avg       0.98      0.98      0.98      1115

